<a href="https://colab.research.google.com/github/astopchatyy/GenAlgoTest/blob/basic_trainer/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import random
import copy
import matplotlib.pyplot as plt
from typing import Dict, List, Type, Iterator
from tqdm import tqdm
from abc import ABC, abstractmethod
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
import os
num_workers = os.cpu_count()
num_workers: int = num_workers if num_workers else 1
print("Number of workers:", num_workers)

Device: cuda
Number of workers: 12


In [3]:
from metric import Accuracy
from trainer import ShuffleTrainer

In [6]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ShuffleResBlock(BasicBlock):
    def __init__(self, planes, expansion=1):
        super().__init__(planes * expansion, planes, stride=1)
        self.expansion = expansion

class ShuffleResNet(nn.Module):
    def __init__(self, num_blocks: List[int], num_classes=10):
        super(ShuffleResNet, self).__init__()
        self.in_planes = 64
        self.block = BasicBlock
        self.shuffle_block_class = ShuffleResBlock

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()

        self.layer1 = self._make_layer(self.block, 128, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(self.block, 256, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(self.block, 512, num_blocks[2], stride=2)

        self.layer4 = self._make_shuffle_layer(ShuffleResBlock, 512, num_blocks[3], stride=1)
        self.layer5 = self._make_shuffle_layer(ShuffleResBlock, 512, num_blocks[4], stride=1)
        self.layer6 = self._make_shuffle_layer(ShuffleResBlock, 512, num_blocks[5], stride=1)


        self.linear = nn.Linear(512 * self.block.expansion, num_classes)

    def _make_layer(self, block: Type[BasicBlock], planes: int, num_blocks: int, stride: int):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride_val in strides:
            layers.append(block(self.in_planes, planes, stride_val))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def _make_shuffle_layer(self, block: Type[ShuffleResBlock], planes: int, num_blocks: int, stride: int):
        layers = []
        for _ in range(num_blocks):
            layers.append(block(planes))
        return nn.Sequential(*layers)


    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)

        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)

        out = nn.functional.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [5]:
class SimpleBlock(nn.Module):
    def __init__(self):
        super(SimpleBlock, self).__init__()
        self.conv = nn.Conv2d(256, 256, 3, padding=1)
        self.dropout1 = nn.Dropout(0.25)

    def forward(self, x):
        x = torch.relu(self.conv(x))
        x = self.dropout1(x)
        return x

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.b1 = SimpleBlock()
        self.b2 = SimpleBlock()
        self.b3 = SimpleBlock()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = self.dropout1(x)

        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)

        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = self.dropout1(x)

        x = torch.relu(self.conv4(x))
        x = self.dropout1(x)

        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)

        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x



In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

base_train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

In [ ]:
validation_fraction = 0.2
validation_size = int(len(testset) * validation_fraction)
seed = 42
g = torch.Generator().manual_seed(seed)
valset, trainset = torch.utils.data.random_split(base_train_set, [validation_size, len(base_train_set) - validation_size], generator=g)

val_loader = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=False, num_workers=num_workers)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(testset, batch_size=128,
                                          shuffle=False, num_workers=num_workers)

In [ ]:
pop_size = 1
trainer = ShuffleTrainer(pop_size, ShuffleResNet, optim.Adam, nn.CrossEntropyLoss(), Accuracy(), [ShuffleResBlock], optimizer_params={"lr": 0.005}, model_params={"num_blocks": [1,1,1,1,1,1]})
history = []

In [ ]:
history.append(trainer.train(1, train_loader, test_loader, val_loader, epoches_per_cycle=200, last_cycle_evolution=False, verbose=1))

In [ ]:
max_vals = [0 for _ in range(pop_size)]
test_vals = [0 for _ in range(pop_size)]
for i in range(pop_size):
    for el in history[-1]["val_metric"][i]:
        for val in el:
            if val > max_vals[i]:
                max_vals[i] = val
                test_vals[i] = history[-1]["test_metric"][i][-1][-1]

print(max_vals)
print(max(max_vals))

print("====")
print(test_vals)
print(test_vals[np.argmax(max_vals)])

[0.8293945305049419]
0.8293945305049419
====
[0.8081487341772152]
0.8081487341772152


In [ ]:
model = trainer.extract_model(0.8)
trainer._evaluate_model(model, test_loader)

0.821993670886076

In [ ]:
for i in range(trainer._genetic_population_size):
  torch.save(trainer._population[i], f"/content/drive/MyDrive/gen_algo/4l/gen/model_4l_gen_{i}")

In [ ]:
for i in range(trainer._genetic_population_size):
  trainer._population[i] = torch.load(f"/content/drive/MyDrive/gen_algo/4l/gen/model_4l_gen_{i}", weights_only=False)

for i in range(trainer._superviser_population_size):
  trainer._population[trainer._genetic_population_size + i] = torch.load(f"/content/drive/MyDrive/gen_algo/4l/sep/model_4l_sep_{i}", weights_only=False)

In [ ]:
pops = []
for i in range(trainer._genetic_population_size):
  model = trainer._population[i]
  pops.append(model)

In [ ]:
for i in range(trainer._genetic_population_size):
  index_a, index_b = random.sample(range(len(pops)), 2)
  trainer._population[i] = trainer._crossover(pops[index_a], pops[index_b], mutation_chance=0.3)

In [ ]:
h = [[],[]]
h[0] = trainer.train(4, train_loader, test_loader, val_loader, epoches_per_cycle=1, last_cycle_evolution=False, mutation_chance=0.4)
h[1] = trainer.train(4, train_loader, test_loader, val_loader, epoches_per_cycle=1, last_cycle_evolution=False, mutation_chance=0)

In [ ]:
model = trainer.extract_model(1)
trainer._evaluate_model(model, test_loader)

0.8269382911392406

In [ ]:
model = trainer.extract_model(1)
trainer._evaluate_model(model, test_loader)

0.8267405063291139

In [ ]:
trainer2 = GeneticTrainer(1, 0, SimpleCNN, optim.Adam, nn.CrossEntropyLoss(), Accuracy(), optimizer_params={"lr": 0.001})
hh = trainer2.train(1, train_loader, test_loader, val_loader, epoches_per_cycle=200, last_cycle_evolution=False, mutation_chance=0)


In [ ]:
print(max(hh["val_metric"][0][0]), np.argmax(hh["val_metric"][0][0]), hh["test_metric"][0][0][np.argmax(hh["val_metric"][0][0])])

0.8433593772351742 195 0.8214003164556962


In [ ]:
[h[1]["test_metric"][i][-1] for i in range(len(h[1]["test_metric"]))]

[[0.8163568037974683],
 [0.8149723101265823],
 [0.8134889240506329],
 [0.811807753164557],
 [0.8158623417721519],
 [0.8145767405063291],
 [0.8084454113924051],
 [0.8166534810126582],
 [0.8089398734177216],
 [0.8143789556962026],
 [0.8137856012658228],
 [0.8194224683544303],
 [0.8082476265822784],
 [0.8157634493670886],
 [0.807060917721519],
 [0.806368670886076]]

In [ ]:
for i in range(trainer2._genetic_population_size):
  torch.save(trainer2._population[i], f"/content/drive/MyDrive/gen_algo/4l/sep/model_4l_sep_{i}")

In [ ]:
for i in range(trainer2._genetic_population_size):
  trainer2._population[i] = torch.load(f"/content/drive/MyDrive/gen_algo/4l/sep/model_4l_sep_{i}", weights_only=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.makedirs("/content/drive/MyDrive/gen_algo/4l/gen", exist_ok=True)
os.makedirs("/content/drive/MyDrive/gen_algo/4l/sep", exist_ok=True)